In [ ]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis')

In [ ]:
classifier('I am very happy as it is the last day of the workshop :)')

In [ ]:
results = classifier(["I am very happy as it is the last day of the workshop :)",
           "I am also sad becuase its ending today"])
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-large")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-large")

# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens,
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))


In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/bert-large-uncased-whole-word-masking-squad2"

# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': 'Why is model conversion important?',
    'context': 'The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.'
}
res = nlp(QA_input)

print (res)

In [ ]:
from transformers import BertTokenizerFast, EncoderDecoderModel
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = BertTokenizerFast.from_pretrained('mrm8488/bert-small2bert-small-finetuned-cnn_daily_mail-summarization')
model = EncoderDecoderModel.from_pretrained('mrm8488/bert-small2bert-small-finetuned-cnn_daily_mail-summarization').to(device)

def generate_summary(text):
    # cut off at BERT max length 512
    inputs = tokenizer([text], padding="max_length", truncation=True, max_length=100, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)

    output = model.generate(input_ids, attention_mask=attention_mask)

    return tokenizer.decode(output[0], skip_special_tokens=True)

text = """In November 2021, nearly 200 countries gathered in Glasgow, Scotland, for the 26th United Nations Climate Change Conference (COP26), a pivotal moment in global efforts to address the climate crisis.
The summit came amid growing scientific evidence that the world is not on track to limit global warming to 1.5°C above pre-industrial levels — the goal set by the 2015 Paris Agreement.
The conference aimed to finalize outstanding elements of the Paris rulebook and push countries to strengthen their emissions-reduction targets.

During the two-week summit, a wide range of negotiations took place. One of the key outcomes was the Glasgow Climate Pact, which called on countries to revisit and strengthen their nationally determined contributions (NDCs) by the end of 2022.
It also included language, for the first time, explicitly urging the phase-down of unabated coal power and the phase-out of inefficient fossil fuel subsidies.
While this language was weakened at the last minute due to pressure from a few coal-dependent countries, its inclusion marked a historic moment in UN climate diplomacy.

Another major focus of COP26 was climate finance. Developing nations, particularly small island states and least developed countries, emphasized the need for wealthy nations to deliver on their long-standing promise of mobilizing $100 billion annually in climate finance to support adaptation and mitigation efforts.
Although this goal was not fully achieved during the summit, developed countries reaffirmed their commitment and pledged increased funding for the years ahead.

The role of non-state actors also gained prominence at COP26. Businesses, cities, and civil society organizations showcased climate initiatives and pushed for more ambitious action.
The summit featured major announcements, including pledges to reduce methane emissions by 30% by 2030, end deforestation by the same year, and accelerate the transition to zero-emission vehicles.
The participation of indigenous communities and youth activists brought attention to the social and environmental justice dimensions of climate change, reminding leaders of the human impacts of their decisions.

Despite some progress, COP26 was met with mixed reactions. Climate scientists and activists warned that the summit did not go far enough to close the gap between current pledges and the reductions needed to keep 1.5°C within reach.
Many observers noted that while incremental steps were taken, more transformative change is urgently needed. The next few years will be critical, as countries must translate promises into policies and actions.

Ultimately, COP26 underscored both the challenges and opportunities of global climate governance. It reaffirmed the urgency of the crisis, highlighted the importance of international cooperation, and reminded the world that time is running out to prevent the most catastrophic effects of climate change.
"""
generate_summary(text)
